In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_predict, plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
temp = pd.read_csv('../input/ajaccio-daily-temperature/ajaccio_daily_temperature.csv')
temp.head()

In [ ]:
temp['original_period'] = pd.to_datetime(temp['original_period'])
temp.set_index('original_period', inplace=True)

In [ ]:
temp.isna().sum()

In [ ]:
temp.fillna(method='bfill', inplace=True)
temp.fillna(temp['original_value'].mean(), inplace=True)

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
plt.style.use('fivethirtyeight')
ax1 = plt.plot(temp)
plt.axhline(temp['original_value'].mean(), ls='--', color='red', label='mean')

plt.fill_between(temp.index, temp['original_value'].mean() + np.std(temp['original_value']),
                 temp['original_value'].mean() - np.std(temp['original_value']),
                 facecolor='red', alpha=0.5, label='std')
plt.legend()
plt.title('Daily temperature Ajaccio (Cº)')
plt.show()

In [ ]:
decompose = seasonal_decompose(temp['original_value'], period=365)

decompose.plot()
plt.show()

In [ ]:
temp = temp.resample("M").mean()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,6))
plt.style.use('fivethirtyeight')
ax1 = plt.plot(temp)
plt.axhline(temp['original_value'].mean(), ls='--', color='red', label='mean')

plt.fill_between(temp.index, temp['original_value'].mean() + np.std(temp['original_value']),
                 temp['original_value'].mean() - np.std(temp['original_value']),
                 facecolor='red', alpha=0.5, label='std')
plt.legend()
plt.title('Daily temperature Ajaccio (Cº)')
plt.show()

In [ ]:
decompose = seasonal_decompose(temp, period=12)

decompose.plot()
plt.show()

In [ ]:
temp.loc['2013':'2015'].plot()
plt.show()

In [ ]:
print("value:", adfuller(temp['original_value'], 12)[0],
     "p-value:", adfuller(temp['original_value'], 12)[1],
     'conf:', adfuller(temp['original_value'], 12)[4])

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(18,6))

ax1 = plot_acf(temp['original_value'], ax=ax1, lags=12, zero=False)
ax2 = plot_pacf(temp['original_value'], ax=ax2, lags=12, zero=False)

plt.show()

In [ ]:
modelo = SARIMAX(temp, order=(0,0,0), seasonal_order=(2,0,0,12))
res = modelo.fit()
print(res.summary())

In [ ]:
res.plot_diagnostics(figsize=(12,6))
plt.show()

In [ ]:
predict = res.get_forecast(steps = 36)
predict_mean = predict.predicted_mean

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.plot(temp)
plt.plot(predict_mean)
plt.show()